# Fine-tuned GPT-3 Article Generation Pipeline
This file shows how we used GPT-3 to generate synthetic news articles after having fine-tuned it for this downstream task. The API key is removed for obvious reasons.

## SETUP 

In [55]:
## load packages for openai
import openai
import os

In [56]:
## api file import ##
openai.api_key = "INSERT KEY"

## alternative way to import## 
#openai.api_key = os.environ["OPENAI_API_KEY"]

### Importing GPT-3

In [57]:
# store all fine-tuned models in a list
finetunes = openai.FineTune.list()

#get the fine-tuned model name
fine_tuned_model = finetunes.data[3].fine_tuned_model

#check if right model is selected
print(fine_tuned_model) 

davinci:ft-personal:finetuning-samf2-2022-10-14-11-09-47


### Load Prompts

In [80]:
import pandas as pd

path = "~/Desktop/politics-GPT3/data/articles (20. sep - 30. oct)/shuffled_final_articles (20. sep - 30. oct).csv"

data = pd.read_csv(path)

In [81]:
#subset data for generation (rest will be used as actual human articles)
data = data[179:]

In [82]:
#create prompt column 
data["prompt"] = data["headers"].astype(str) +". "+ data["sub_header"] +" ->"

## GENERATION PIPELINE

In [95]:
data1 = data.loc[179:238]
data2 = data.loc[239:298]
data3 = data.loc[299:358]

60
60
59


In [60]:
prompt_lst = []
completion_lst = []

In [104]:

for i in range(0,60):
    prompt = data3["prompt"][i+299]
    prompt_lst.append(prompt)

    completion = openai.Completion.create(
    model=fine_tuned_model,
    prompt=prompt, 
    stop = "\n", 
    temperature = 0,
    max_tokens=400,
    best_of=1,
    frequency_penalty=0.2,
    presence_penalty=0.2,
)
    completion_lst.append(completion.choices[0].text)

KeyError: 358

In [105]:
gpt3_data = pd.DataFrame({"prompt": prompt_lst, "completion": completion_lst})
gpt3_data

,prompt,completion
0,København vil inkludere LGBT-personer bedre me...,Københavns Kommune vil fremover øve sig i at ...
1,Sygeplejersker på efterløn overvejer at vende ...,De seneste ugers store demonstrationer og de ...
2,København vil inkludere LGBT-personer bedre me...,Københavns Kommune vil fremover øve sig i at ...
3,Sygeplejersker på efterløn overvejer at vende ...,De seneste ugers store demonstrationer og de ...
4,Dømt for landsforræderi sætter punktum for sin...,"En 33-årig mand, der i maj 2020 blev dømt for..."
...,...,...
176,Hendes børns klub lukker - det kan ske for and...,"En børneidrætsklub i København må lukke, ford..."
177,Susanne Christensens mor ventede otte måneder ...,"Susanne Christensen var i sit livs form, da h..."
178,Danskerne flytter større del af elforbruget ti...,De danske husejere flytter mere og mere af de...
179,Dansk Oscar-kandidat smidt ud af Tyrkiet og fo...,"Det er ikke alle, der kan lide den danske fil..."


In [106]:
gpt3_data.to_csv("all_gpt3_articles.csv")